# Running Computations in Parallel


What would be simplest way to indicate two expression `e1` and `e2` to run in parallel?

`parallel(e1,e2)`

To understand this construct let's look at an example.

Given a vector as an array (of integers), compute its `p-norm`

A `p-norm` is a generalization of the notion of length from geometry

2-norm of a two-dimentional vector $(a_1, a_2)$ is ${(a_1^2 + a_2^2)}^{\frac{1}{2}}$

The p-norm of a vector $(a_1,a_2,\dots,a_n)$ is $(\sum_{i=1}^{n}{|a_i|}^p)^{\frac{1}{p}}$

### Main step: sum of powers of array segment

First, solve sequentially the following `sumSegment` problem: given
* an integer array $a$, representing our vector
* a positive double floating point number $p$
* two valid indices $s \leq t$ into the array a
compute

$(\sum_{i=s}^{t-1}{\left \lfloor{{|a_i|}^p}\right \rfloor })^{\frac{1}{p}}$
where $⌊y⌋$ rounds down to an integer
 
The main step in the solution is to compute the sum of the elements the array raised to p. Let us define a slightly more general function, called `sumSegment`. It should take an array $a$ and the number $p$ (which we represent as a double), but also the start index of the segment as well as the end boundary, an index before which we should stop summing up.

Here power computes :

$\left \lfloor{{|x|}^p}\right \rfloor $

In [1]:
def power(x: Int, p: Double): Int = math.exp(p * math.log(math.abs(x))).toInt

def sumSegment(a: Array[Int], p: Double, s: Int, t: Int): Int = {
var i= s; var sum: Int = 0
while (i < t) {
sum= sum + power(a(i), p)
i= i + 1
}
sum
}

defined function power
defined function sumSegment

Given `sumSegment(a,p,s,t)`, how to compute `p-norm`?

$||a||_p := (\sum_{i=1}^{N-1}{|a_i|}^p)^{\frac{1}{p}}$

$N=a.length$

We give to `sumSegment` the entire array, from index zero to the length of the array. 
We then raise the result to the power one over $p$. This gives us a
sequential version for computing the p-norm. 


In [2]:
def pNorm(a: Array[Int], p: Double): Int =
power(sumSegment(a, p, 0, a.length), 1/p)

defined function pNorm

How do we go from here to a parallel version?

Now, observe that the sumation can be expressed in two parts: sum up to some middle element m, and then sum from that middle element to the end.

$||a||_p := (\sum_{i=1}^{N-1}{|a_i|}^p)^{\frac{1}{p}} =  (\sum_{i=1}^{m-1}{|a_i|}^p + \sum_{i=m}^{N-1}{|a_i|}^p)^{\frac{1}{p}}$
What is a Scala expression that corresponds to using two sums?

In [3]:
def pNormTwoPart(a: Array[Int], p: Double): Int = {
val m = a.length / 2
val (sum1, sum2) = (sumSegment(a, p, 0, m),
sumSegment(a, p, m, a.length))
power(sum1 + sum2, 1/p) }

defined function pNormTwoPart

All we need to do is invoke sum segment twice, then add up the two intermediate sums before raising everything to the power of one over p. This is still
sequential computation. How do we make it parallel?

All we need to is insert `parallel` combinator.

```scala

val (sum1, sum2) = parallel(sumSegment(a, p, 0, m),
sumSegment(a, p, m, a.length))

```

How do we generalize this to proces four segments in parallel?

```scala
val m1 = a.length/4; val m2 = a.length/2; val m3 = 3*a.length/4
val ((sum1, sum2),(sum3,sum4)) =
parallel(parallel(sumSegment(a, p, 0, m1), sumSegment(a, p, m1, m2)),
parallel(sumSegment(a, p, m2, m3), sumSegment(a, p, m3, a.length)))
```
Is there a recursive algorithm for an unbounded number of threads?

We have seen how to run summation over two or four segments in parallel. Now, suppose that we have a very long array and an essentially unbounded
number of parallel hardware resources. Is there an algorithm that spawns as many parallel computations as needed?

```scala
def pNormRec(a: Array[Int], p: Double): Int =
power(segmentRec(a, p, 0, a.length), 1/p)


// like sumSegment but parallel
def segmentRec(a: Array[Int], p: Double, s: Int, t: Int) = {
if (t - s < threshold)
sumSegment(a, p, s, t) // small segment: do it sequentially
else {
val m = s + (t - s)/2
val (sum1, sum2) = parallel(segmentRec(a, p, s, m),
segmentRec(a, p, m, t))
sum1 + sum2 } }

```

But what is the signature of `parallel`?

```scala

def parallel[A, B](taskA: => A, taskB: => B): (A, B) = { ... }

```
* returns the same value as given
* benefit: `parallel(a,b)` can be faster than `(a,b)`
* it takes its arguments as by name, indicated with `=> A` and `=> B`

Here is the type signature of parallel. It takes the two computations as parameters, here denoted `taskA` and `taskB`. It returns a pair of `A` and `B`, storing the
values of those computations. From the point of view of result, parallel behaves as an identity function. The benefit is, of course, that the computation
can finish sooner than computing first `taskA` and then `taskB`. Note that the types of parameters are declared as arrow `A` and arrow `B`. Why not simply `A` and `B`?

To help answer this question, consider a seemingly minor variant of parallel called parallel1, which differs from parallel only in that it takes parameters by
value instead of by name. How does parallel1 behave? If we have two long-running computations a and b, what is the difference between: parallel of a
and b versus: parallel1 of a and b?

Suppose that both parallel and parallel1 contain the same body:
```scala
def parallel [A, B](taskA: => A, taskB: => B): (A, B) = { ... }
def parallel1[A, B](taskA: A, taskB: B): (A, B) = { ... }
```
If `a` and `b` are some expressions what is the difference between

1. 

```scala
val (va, vb) = parallel(a, b)
```

2. 

```scala
val (va, vb) = parallel1(a, b) 
```

In fact, the second computation does not use parallelism at all. Because the arguments to `parallel1` are passed by value, they are first evaluated, one by one, and this is where the time is spent. 

Then their values are passed to parallel, which returns those same two values without doing any useful work. 

It is therefore essential that we do not evaluate the computation before giving it to a construct that runs it in parallel. For this reason it is appropriate to use call by name parameters, indicated by the arrow in the signature of parallel. 

Like expressions if and while, parallel is a control structure.

We have seen how to use the parallel construct. What happens behind the scenes when we invoke computations in parallel? 

To support parallelism efficiently, we need support from different layers of a computing system: 

* the language and libraries, 

* virtual machine (such as Java Virtual Machine),

* the operating system, 

* and the hardware itself. 

One implementation of parallel uses Java’s threads. On most platforms these are mapped to threads of the underlying operating system. 
The operating system provides ability to run many threads and processes. 
When the underlying hardware has multiple
processor cores, these threads can execute on different cores, which results in parallel execution. 

Thanks to the flexibility in different layers of the software
stack, a program written with parallelism in mind will run even when there is only one processor core available (of course, without the speedup).

As a result of this complexity, performance of parallel code is affected by some of the hardware arhitecture aspects. To illustrate this, consider the following
`sum`1 function, which is like `sumSegment`, but sums up array elements themselves instead of their powers. Suppose that we try to execute four such sums in parallel on a commodity desktop with, say, four physical cores.


```scala

def sum1(a: Array[Int], p: Double, s: Int, t: Int): Int = {
var i= s; var sum: Int = 0
while (i < t) {
sum= sum + a(i) // no exponentiation!
i= i + 1
}
sum }
val ((sum1, sum2),(sum3,sum4)) = parallel(
parallel(sum1(a, p, 0, m1), sum1(a, p, m1, m2)),
parallel(sum1(a, p, m2, m3), sum1(a, p, m3, a.length)))
```

We may find it difficult to get any notable speedup, despite the fact that we do get speedups if we run the computation that does the more expensive operation per array. What is more, this problem remains even if we make the size of the array very large. What is happening?

It turns out that this computation is bound by the memory bandwidth. The array is stored in random access memory. Whether we have one or more
cores working, the cores spend their time waiting for the elements of the array to be fetched from the random access memory. Even though computations happen in parallel, we need to take into consideration of parallelism of shared resources.

Recall that we have previously ran in parallel computations on array segments of approximately same length, resulting in similar time for different parallel
threads. 

What happens when we invoke parallel on two computations that take very different time to execute?

We need to wait for both e1 and e2 to complete execution before we can return the pair of values. Therefore, the running time, in the best case, is the
maximum of the two running times.

```scala
val (v1, v2) = parallel(e1, e2)
```
The running time of `parallel(e1, e2)` is the maximum of two running times.